### Data Source: 

#### Query
Web of Science Query = (

  TS=(biodivers* OR "biological diversity" OR "bio-divers*" OR "bio divers*")

  OR SU="Biodiversity & Conservation"

)

AND DT=(Article OR Review)

AND PY=(1980-2025)

#### Query Notes
- Period: Starting 1980s since this is period biodiversity as field emerged

- SU = Research Area in WOS: https://webofscience.help.clarivate.com/Content/current-contents/ccc-search-field-tags.htm

In WOS research areas (total 5), Life Science & Biomedicine (1/5) contains only this tag for biodiversity: "Biodiversity & Conservation"

- TS = Topic Search = [Title, Abstract, Author Keywords, Keywords Plus]

Topic search query is kept general here and adopted from 2016 bibliometric assessment here: (See page 2)
https://www.scirp.org/pdf/OJE_2014052617163441.pdf#page=12.10




#### Notes 

- WOS Results returned = 359,435 records

- Time of most recent query: 10 Jul 2025 311pm - 11pm 

- Internal note: Version 2 of query


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Dataframes

In [1]:
df = pd.read_excel("dataset/curated_data/data.xlsx")
df_sample = df.sample(n=1000, random_state=42)
df_sample.to_csv("dataset/curated_data/sample_data.csv", index=False)
core_cols = [
    "Publication Type", "Article Title", "Language", "Document Type",
    "Author Keywords", "Keywords Plus", "Abstract", 
    "Publisher", "Publisher City", "Publication Date", "Publication Year", 
    "Web of Science Index", "WoS Categories", "Research Areas", 
    "Highly Cited Status", "Hot Paper Status", "UT (Unique WOS ID)"
]


NameError: name 'pd' is not defined

In [ ]:
df_core = df[core_cols].copy()

### 2. Overall Summary

In [ ]:
cols = df.columns

# Count summary
    # Row counts
count_summary = {
    "row_count": len(df_core),
    "web_of_science_record_count": df_core["UT (Unique WOS ID)"].count(),
    "web_of_science_record_distinct_count": df_core["UT (Unique WOS ID)"].nunique(),
    "duplicated_row_count": df_core.duplicated().sum()
}
    # Nulls
null_counts = df_core.isnull().sum()
null_cols = null_counts[null_counts > 0].sort_values(ascending=False)

#### - Shape & Core Cols

- Core cols are subset of cols useful for analysis 
- Overall Counts:
    - Source Data Count = 359,435
    - Duplicates = 502
    - Count after dropping duplicates = 358,933

In [ ]:
print(f"Dataset shape: {df_core.shape}")
print(f"{len(core_cols)} Core Cols out of {len(cols)} total cols in the dataset:")
print(core_cols)
print(f" Count Summary : {count_summary}")



In [ ]:
df_core = df_core.drop_duplicates()
print(f"Dropped {count_summary['row_count'] - len(df_core)} duplicated rows. Remaining rows: {len(df_core)}")

#### - Nulls
- ~16k records contains no abstract. There are dropped from df_core
- Original Row count = 359,435; New Row count: 342,484

In [ ]:
print(f" {len(null_cols)} Columns with null values:   {null_cols}" )

In [ ]:
before_count = len(df_core)
df_core = df_core.dropna(subset=["Abstract"])
after_count = len(df_core)
print(f"Dropped {before_count - after_count} rows. Remaining rows: {after_count}")

#### - Distinct Value Counts
- Nulls: Expected to have nulls in "Highly Cited"  and "Hot Paper" since it's "Y or N" only when applicable. Else null.
- Keywords are not always given (Keyword plus , author keyword) - ~15-20% rows
- Basic details like title, abstract, categories, researach area have no or 1,2 nulls only

In [ ]:
distinct_counts = df_core.nunique(dropna=True).sort_values(ascending=False)
null_counts = df_core.isnull().sum()
summary = pd.DataFrame({'distinct': distinct_counts, 'nulls': null_counts[distinct_counts.index]})
print(summary)

#### - Language
- ~97.7% of publications are in "English"

In [ ]:
lang_counts = df_core['Language'].value_counts(dropna=False)
lang_percent = df_core['Language'].value_counts(normalize=True, dropna=False) * 100
lang_summary = pd.DataFrame({'count': lang_counts, 'percent': lang_percent.round(2)})
print(lang_summary)

### S2

In [ ]:
year_counts = df_core['Publication Year'].value_counts().sort_index()
plt.figure(figsize=(12,6))
plt.bar(year_counts.index, year_counts.values / 1000)
plt.xlabel('Publication Year')
plt.ylabel('Count of Records (Thousands)')
plt.title('Publication count by year of publication')
plt.xticks(year_counts.index, rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
top_research_areas = df_core['Research Areas'].value_counts().head(15)

plt.figure(figsize=(12, 6))
plt.barh(top_research_areas.index[::-1], top_research_areas.values[::-1]/ 1000)  # reverse for top-to-bottom
plt.title("Top 15 Research Areas by Number of Publications")
plt.xlabel('Count of Records (Thousands)')
plt.ylabel("Research Area")
plt.tight_layout()
plt.show()

In [3]:
import pandas

In [5]:
from matplotlib_venn import venn3

ModuleNotFoundError: No module named 'matplotlib_venn'

In [ ]:

# from matplotlib_venn import venn3


# # Load the data
# df = pd.read_csv("sample_data.csv")

# # Prepare the Research Areas column
# research_series = df['Research Areas'].dropna()
# split_research_areas = research_series.str.split(';').apply(lambda x: [i.strip() for i in x])

# # Create binary indicator matrix
# mlb = MultiLabelBinarizer()
# indicator_matrix = pd.DataFrame(mlb.fit_transform(split_research_areas),
#                                 columns=mlb.classes_,
#                                 index=split_research_areas.index)

# # Get top 3 research areas
# top_3_areas = indicator_matrix.sum().sort_values(ascending=False).head(3).index.tolist()

# # Create sets for each area
# set1 = set(indicator_matrix[indicator_matrix[top_3_areas[0]] == 1].index)
# set2 = set(indicator_matrix[indicator_matrix[top_3_areas[1]] == 1].index)
# set3 = set(indicator_matrix[indicator_matrix[top_3_areas[2]] == 1].index)

# # Plot Venn Diagram
# plt.figure(figsize=(8, 6))
# venn3([set1, set2, set3], set_labels=top_3_areas)
# plt.title("Overlap of Top 3 Research Areas")
# plt.show()